In [1]:
from collections import defaultdict
from pathlib import Path
import ylekone

In [2]:
class Stats:
    # Don't bother showing counts of less than this
    THRESHOLD = 0
    
    def __init__(self):
        self._total = 0
        self._data = defaultdict(lambda: 0)
        self._langs = defaultdict(lambda: 0)
        self._parties = defaultdict(lambda: 0)

    @property
    def parties(self):
        return Stats._keys_by_count(self._parties, self.THRESHOLD)

    @property
    def languages(self):
        return Stats._keys_by_count(self._langs, self.THRESHOLD)
        
    def add(self, party, lang):
        self._total += 1
        self._data[f"{party}:{lang}"] += 1
        self._langs[lang] += 1
        self._parties[party] += 1

    def get(self, party, lang):
        return self._data.get(f"{party}:{lang}", 0)

    @staticmethod
    def _keys_by_count(d, threshold):
        d = [(v, k) for k, v in d.items() if v > threshold]
        d = sorted(d)[::-1]
        return [k for v, k in d]

In [3]:
def append_stats(stats, constituency):
    for candidate in constituency.candidates:
        if candidate.party is not None:        
            stats.add(candidate.party.abbreviation, candidate.language)
        else:
            stats.add("??", candidate.language)

def cstats(constituency):
    stats = Stats()
    append_stats(stats, constituency)
    return stats

def astats(constituencies):
    stats = Stats()
    for constituency in constituencies:
        append_stats(stats, constituency)
    return stats

In [4]:
def mktable(stats):
    s = ""
    langs = stats.languages
    parties = stats.parties
    s += " " * 8
    for lang in langs:
        s += f"{lang:>16}"
    s += "\n"
    for party in parties:
        s += f"{party:>8}"
        for lang in langs:
            s += f"{stats.get(party, lang):>16}"
        s += "\n"
    return s

In [5]:
def wmd(name, stats, outfile):
    outfile = Path(outfile)
    outfile.parent.mkdir(exist_ok=True)
    with open(outfile, "w", encoding="utf-8") as f:
        f.write(f"# Ehdokkaiden kielet, {name}\n\n")
        f.write(f"| |yhteensä|")
        for lang in stats.languages:
            f.write(f"{lang}|")
        f.write(f"\n")
        
        f.write(f"|:---:|:---:|")
        for lang in stats.languages:
            f.write(f":---:|")
        f.write(f"\n")
        
        f.write(f"|Yhteensä|")
        f.write(f"{stats._total}|")
        for lang in stats.languages:
            count = stats._langs[lang]
            pct = 100 * count / stats._total
            if count == 0:
                f.write(f"{count}|")
            elif pct < 10:
                f.write(f"{count} ({pct:.2f}%)|")
            else:
                f.write(f"{count} ({pct:.1f}%)|")
        f.write(f"\n")
        
        for party in stats.parties:
            total = stats._parties[party]
            f.write(f"|{party}|{total}|")
            for lang in stats.languages:
                count = stats.get(party, lang)
                pct = 100 * count / total
                if count == 0:
                    f.write(f"{count}|")
                elif pct < 10:
                    f.write(f"{count} ({pct:.2f}%)|")
                else:
                    f.write(f"{count} ({pct:.1f}%)|")
            f.write("\n")
            
        f.write("\n")

In [6]:
constituencies = ylekone.kv25()
stats_all = astats(constituencies)
stats_ind = {}
for constituency in constituencies:
    stats_ind[constituency.name] = cstats(constituency)

In [7]:
wmd("koko maa", stats_all, "kielet/kuntavaalit/koko_maa.md")
for k, v in stats_ind.items():
    wmd(k, v, f"kielet/kuntavaalit/{k}.md")

In [8]:
constituencies = ylekone.av25()
stats_all = astats(constituencies)
stats_ind = {}
for constituency in constituencies:
    stats_ind[constituency.name] = cstats(constituency)

In [9]:
wmd("koko maa", stats_all, "kielet/aluevaalit/koko_maa.md")
for k, v in stats_ind.items():
    wmd(k, v, f"kielet/aluevaalit/{k}.md")